In [1]:
import pandas as pd

overdose = pd.read_csv('Overdose.csv')

#Filter for years after or at 2018
overdose = overdose[overdose["Year"] >= 2018] 

#Overdoses by years 2018, 2019, 2020
overdose = overdose.groupby(['County', 'Year'])["Count"].sum().reset_index()
overdose = overdose.rename(columns={'Count':'Overdoses'})
year18 = overdose[overdose["Year"] == 2018] 
year19 = overdose[overdose["Year"] == 2019] 
year20 = overdose[overdose["Year"] == 2020] 
overdose['2018 Overdoses'] = year18["Overdoses"]
overdose['2019 Overdoses'] = year19["Overdoses"]
overdose['2020 Overdoses'] = year20["Overdoses"]
overdose = overdose.drop(columns = ["Year", "Overdoses"]).groupby("County").sum().reset_index()
overdose.head(100)

,County,2018 Overdoses,2019 Overdoses,2020 Overdoses
0,Alamance,168699.0,171740.0,173697.0
1,Alexander,37995.0,37929.0,37900.0
2,Alleghany,11298.0,11257.0,11359.0
3,Anson,25336.0,24749.0,24541.0
4,Ashe,27479.0,27550.0,27488.0
...,...,...,...,...
95,Wayne,124861.0,125185.0,125625.0
96,Wilkes,69913.0,69570.0,69087.0
97,Wilson,82272.0,82789.0,83295.0
98,Yadkin,38310.0,38211.0,38232.0


In [2]:
income = pd.read_csv('lapi1121_formatted.csv')

income['2018'] = income['2018'].str.split(',').str.join('').astype(int)
income['2019'] = income['2019'].str.split(',').str.join('').astype(int)
income['2020'] = income['2020'].str.split(',').str.join('').astype(int)
income = income.rename(columns={'Region': 'County', '2018' : '2018 Per Capita Income', '2019' : '2019 Per Capita Income', '2020' : '2020 Per Capita Income'})
income.head(100)

,County,2018 Per Capita Income,2019 Per Capita Income,2020 Per Capita Income
0,Alamance,39742,41256,43973
1,Alexander,36539,37758,40176
2,Alleghany,36576,37987,40599
3,Anson,35135,37741,38785
4,Ashe,34791,36643,39612
...,...,...,...,...
95,Wayne,38115,40116,42882
96,Wilkes,36445,37309,39518
97,Wilson,39158,40260,43111
98,Yadkin,37328,39012,41250


In [3]:
#merge together for our combined dataset
data = pd.merge(overdose, income, on = "County")

#Convert float to int
data['2018 Overdoses'] = data['2018 Overdoses'].astype(int)
data['2019 Overdoses'] = data['2019 Overdoses'].astype(int)
data['2020 Overdoses'] = data['2020 Overdoses'].astype(int)

data.head(100)

,County,2018 Overdoses,2019 Overdoses,2020 Overdoses,2018 Per Capita Income,2019 Per Capita Income,2020 Per Capita Income
0,Alamance,168699,171740,173697,39742,41256,43973
1,Alexander,37995,37929,37900,36539,37758,40176
2,Alleghany,11298,11257,11359,36576,37987,40599
3,Anson,25336,24749,24541,35135,37741,38785
4,Ashe,27479,27550,27488,34791,36643,39612
...,...,...,...,...,...,...,...
95,Wayne,124861,125185,125625,38115,40116,42882
96,Wilkes,69913,69570,69087,36445,37309,39518
97,Wilson,82272,82789,83295,39158,40260,43111
98,Yadkin,38310,38211,38232,37328,39012,41250


In [8]:
pop = pd.read_csv('county_estimates_2020.csv', sep = ";")
pop = pop.drop(columns = ["Estimate", "geom", "centroid"])
pop["Year"] = pd.DatetimeIndex(pop["Year"]).year
pop = pop.sort_values(["County", "Year"]).reset_index().drop(columns = "index")


#If year and county is the same, take the latest reported data for that year
pop = pop.drop_duplicates(subset = ['County','Year'], keep = 'last')

#rearrangement
pop = pop[pop["Year"] >= 2018] 
year18 = pop[pop["Year"] == 2018] 
year19 = pop[pop["Year"] == 2019] 
year20 = pop[pop["Year"] == 2020] 

pop = pop.drop(columns = "Year")
pop['2018 Population'] = year18["value"]
pop['2019 Population'] = year19["value"]
pop['2020 Population'] = year20["value"]
pop = pop.drop(columns = "value")
#overdose = overdose.drop(columns = ["Year", "Overdoses"]).groupby("County").sum().reset_index()

pop = pop.groupby("County").sum().reset_index()


#Convert to int
pop['2018 Population'] = pop['2018 Population'].astype(int)
pop['2019 Population'] = pop['2019 Population'].astype(int)
pop['2020 Population'] = pop['2020 Population'].astype(int)

pop.head(100)

,County,2018 Population,2019 Population,2020 Population
0,Alamance,167407,169785,171980
1,Alexander,36664,36565,36444
2,Alleghany,10924,10919,10867
3,Anson,23075,21999,22030
4,Ashe,26591,26645,26577
...,...,...,...,...
95,Wayne,117851,117947,117333
96,Wilkes,66445,66189,65827
97,Wilson,78585,78991,78784
98,Yadkin,37251,37247,37187


In [10]:
#Combine pop with data
data["2018 Overdoses Per Capita"] = data["2018 Overdoses"] / pop["2018 Population"]
data["2019 Overdoses Per Capita"] = data["2019 Overdoses"] / pop["2019 Population"]
data["2020 Overdoses Per Capita"] = data["2020 Overdoses"] / pop["2020 Population"]
data = pd.merge(data, pop)
data = data.round(decimals = 3)


#Final Dataset
finalData = data
finalData.head(100)

,County,2018 Overdoses,2019 Overdoses,2020 Overdoses,2018 Per Capita Income,2019 Per Capita Income,2020 Per Capita Income,2018 Overdoses Per Capita,2019 Overdoses Per Capita,2020 Overdoses Per Capita,2018 Population,2019 Population,2020 Population
0,Alamance,168699,171740,173697,39742,41256,43973,1.008,1.012,1.010,167407,169785,171980
1,Alexander,37995,37929,37900,36539,37758,40176,1.036,1.037,1.040,36664,36565,36444
2,Alleghany,11298,11257,11359,36576,37987,40599,1.034,1.031,1.045,10924,10919,10867
3,Anson,25336,24749,24541,35135,37741,38785,1.098,1.125,1.114,23075,21999,22030
4,Ashe,27479,27550,27488,34791,36643,39612,1.033,1.034,1.034,26591,26645,26577
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Wayne,124861,125185,125625,38115,40116,42882,1.059,1.061,1.071,117851,117947,117333
96,Wilkes,69913,69570,69087,36445,37309,39518,1.052,1.051,1.050,66445,66189,65827
97,Wilson,82272,82789,83295,39158,40260,43111,1.047,1.048,1.057,78585,78991,78784
98,Yadkin,38310,38211,38232,37328,39012,41250,1.028,1.026,1.028,37251,37247,37187
